In [1]:
import pandas as pd
import numpy as np
from time import strptime
from datetime import datetime, timedelta
from time import mktime
import time

In [2]:
flights_df = pd.read_csv("~/Desktop/DataDump/TestData.csv")

In [3]:
min_date = "2020-01-01 00:00:00"
max_date = "2020-01-07 23:59:59"

In [4]:
flight_dates = pd.to_datetime(flights_df["fl_date"])

In [5]:
flights_df = flights_df[(flight_dates>= min_date) & (flight_dates<= max_date)]

In [6]:
print(min(flights_df["fl_date"]))
print(max(flights_df["fl_date"]))

2020-01-01 00:00:00.000
2020-01-07 00:00:00.000


In [7]:
flights_df.shape

(150623, 20)

In [8]:
flights_df.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance'],
      dtype='object')

In [9]:
WeatherEvents = pd.read_csv("~/Desktop/WeatherEvents_Jan2016-Dec2020.csv")

In [10]:
WeatherEvents.shape

(6274206, 13)

In [11]:
WeatherEvents.head()

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode
0,W-1,Snow,Light,2016-01-06 23:14:00,2016-01-07 00:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
1,W-2,Snow,Light,2016-01-07 04:14:00,2016-01-07 04:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
2,W-3,Snow,Light,2016-01-07 05:54:00,2016-01-07 15:34:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
3,W-4,Snow,Light,2016-01-08 05:34:00,2016-01-08 05:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0
4,W-5,Snow,Light,2016-01-08 13:54:00,2016-01-08 15:54:00,US/Mountain,K04V,38.0972,-106.1689,Saguache,Saguache,CO,81149.0


In [12]:
WeatherEvents["AirportCode"].value_counts()

K3TH    11142
KMLP    10883
KHYW     9268
K0CO     8242
KSMP     7996
        ...  
KCZZ       43
K4HV       27
KCQV       25
KSPL        8
KI39        1
Name: AirportCode, Length: 2071, dtype: int64

In [ ]:
flights_df.columns

In [ ]:
WeatherEvents["Type"].value_counts()

In [ ]:
WeatherEvents["Severity"].value_counts()

### Steps to merge the data
* Filter weather to the min/max interval of the flights data
    * Add columns with local time instead of (UTC) - US daylight saving time is Mar-Nov so no need to do check that
* Create columns for Rain, Fog, Snow, Cold, Precipitation, Storm, and Hail for Arrival and Departure
* For each row in Flights:
    * Get all rows in Weather which contain departure and arrival time and locations
    * Populate the weather columns with weather information
    

### Filtering Weather

In [13]:
origin = [a.split(',', 1)[0] for a in flights_df["origin_city_name"]]
flights_df["origin_city"] = [a.split('/', 1)[0] for a in origin]

destination = [a.split(',', 1)[0] for a in flights_df["dest_city_name"]]
flights_df["dest_city"] = [a.split('/', 1)[0] for a in destination]

In [14]:
all_locations  = set(list(flights_df["origin_city"]) + list(flights_df["dest_city"]))
WeatherEvents = WeatherEvents[WeatherEvents["City"].isin(all_locations)]

In [15]:
all_weather = set(WeatherEvents["City"])
not_in_weather = all_locations - all_weather
len(not_in_weather)# 78 locations missing in Weather

77

In [16]:
len(all_locations)

343

In [17]:
flights_df.shape

(150623, 22)

In [18]:
#Only getting cities that have weather data
flights_df = flights_df[flights_df["origin_city"].isin(all_weather) | flights_df["dest_city"].isin(all_weather)]

In [19]:
flights_df.shape

(148466, 22)

In [20]:
150623 - 148466 # lost 2067, not a huge number

2157

### Add local time columns

![](TimeZones.png)

In [21]:
WeatherEvents["TimeZone"].value_counts()

US/Eastern     548470
US/Central     510497
US/Mountain    135529
US/Pacific     114029
Name: TimeZone, dtype: int64

In [22]:
n = WeatherEvents.shape[0]

In [ ]:
def local_time(UTC_string, time_zone, format_in = "%Y-%m-%d %H:%M:%S"):
    """
    Returns local time given string containing UTC datetime, timezone and date format from string
    Time zones must be one of the following
    US/Central
    US/Eastern
    US/Mountain
    US/Pacific
    """
    time_temp = strptime(UTC_string, format_in)
    time_unix = round(time.mktime(time_temp))
    if time_zone == "US/Central":
        unix_out = time_unix - 3600*6
    elif time_zone == "US/Eastern":
        unix_out = time_unix - 3600*5
    elif time_zone == "US/Mountain":
        unix_out = time_unix - 3600*7
    elif time_zone == "US/Pacific":
        unix_out = time_unix - 3600*8
    else:
        print("Time zone not one of the specified types (see function description)")
    time_out = datetime.fromtimestamp(unix_out).strftime('%Y-%m-%d %H:%M:%S')
    return(time_out)
    

In [ ]:
LocalTimeZoneStart = []
LocalTimeZoneEnd = []
for i in WeatherEvents.index:
    TimeZone_temp = WeatherEvents["TimeZone"][i]
    start_temp = WeatherEvents['StartTime(UTC)'][i]
    end_temp = WeatherEvents['EndTime(UTC)'][i]
    LocalTimeZoneStart.append(local_time(start_temp, TimeZone_temp))
    LocalTimeZoneEnd.append(local_time(end_temp, TimeZone_temp))


In [ ]:
WeatherEvents["LocalTimeStart"] = LocalTimeZoneStart
WeatherEvents["LocalTimeEnd"] = LocalTimeZoneEnd

In [ ]:
WeatherEvents.head()

In [ ]:
after_start_date = WeatherEvents["LocalTimeEnd"] >= min_date
before_end_date = WeatherEvents["LocalTimeStart"] <= max_date
between_two_dates = after_start_date & before_end_date
WeatherEvents = WeatherEvents.loc[between_two_dates]

I have dropped 2 rows as outlier. They were 2 events going on for months

In [ ]:
WeatherEvents.shape #4586 events for cities of interest within the first week of 2020

In [ ]:
WeatherEvents.to_csv("~/Desktop/LHL---midterm/mid-term-project-I/data/Weather_FirstWeek2020.csv")

### Mergind both data

In [23]:
WeatherEvents = pd.read_csv("~/Desktop/LHL---midterm/mid-term-project-I/data/Weather_FirstWeek2020.csv", index_col = "Unnamed: 0")
WeatherEvents.reset_index(inplace=True,drop = True)
WeatherEvents["Type"] = WeatherEvents.Type.astype("string")
WeatherEvents["Severity"] = WeatherEvents.Severity.astype("string")
WeatherEvents["LocalTimeStart"] = [datetime.strptime(x,"%Y-%m-%d %H:%M:%S") for x in WeatherEvents["LocalTimeStart"]]
WeatherEvents["LocalTimeEnd"] = [datetime.strptime(x,"%Y-%m-%d %H:%M:%S") for x in WeatherEvents["LocalTimeEnd"]]
flights_df.reset_index(inplace = True, drop = True)

In [24]:
flights_df.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance',
       'origin_city', 'dest_city'],
      dtype='object')

In [25]:
WeatherEvents.columns

Index(['EventId', 'Type', 'Severity', 'StartTime(UTC)', 'EndTime(UTC)',
       'TimeZone', 'AirportCode', 'LocationLat', 'LocationLng', 'City',
       'County', 'State', 'ZipCode', 'LocalTimeStart', 'LocalTimeEnd'],
      dtype='object')

#### Adding departure and arrival date-time column

In [26]:
weather_df = WeatherEvents

In [27]:
n = flights_df.shape[0]

In [28]:
crs_dep_time = []
crs_arr_time = []
for i in range(n):
    dep_time = str(flights_df["crs_dep_time"][i])
    arr_time = str(flights_df["crs_arr_time"][i])
    while len(dep_time)<4:
        dep_time = "0" + dep_time
    while len(arr_time)<4:
        arr_time = "0" + arr_time
    if dep_time == "2400":
        dep_time = "0000"
    if arr_time == "2400":
        arr_time = "0000"
    crs_dep_time.append(dep_time)
    crs_arr_time.append(arr_time)

In [29]:
date_time_dep = [flights_df["fl_date"][x][0:10] + " " + crs_dep_time[x] for x in range(n)]
flights_df["dep_datetime"] = [datetime.strptime(x,'%Y-%m-%d %H%M') for x in date_time_dep]

In [30]:
date_time_arr = [flights_df["fl_date"][x][0:10] + " " + crs_arr_time[x] for x in range(n)]
flights_df["arr_datetime"] = [datetime.strptime(x,'%Y-%m-%d %H%M') for x in date_time_arr]

In [31]:
for i in range(n):
    if int(crs_dep_time[i])>int(crs_arr_time[i]) and (flights_df.at[i,"arr_datetime"].date() == flights_df.at[i,"dep_datetime"].date()):
        #print(flights_df.at[i,"dep_datetime"])
        #print(flights_df.at[i,"arr_datetime"])
        #print("===============================")
        flights_df.at[i,"arr_datetime"] = flights_df["arr_datetime"][i] + timedelta(days=1)


In [32]:
WeatherEvents["Type"].value_counts()

Rain             2567
Snow              960
Fog               921
Cold               55
Storm              44
Precipitation      37
Hail                2
Name: Type, dtype: Int64

In [33]:
def merge_flights_weather(flights_df, weather_df):
    """
    Function designed to merge specific weather and flights data of this project. 
    It will return the flights data with the merged weather info.
    """
    flights_df["dep_Rain"] = "NW"
    flights_df["dep_Fog"] = "NW"
    flights_df["dep_Snow"] = "NW"
    flights_df["dep_Cold"] = "NW"
    flights_df["dep_Storm"] = "NW"
    flights_df["dep_Hail"] = "NW"
    flights_df["dep_Precipitation"] = "NW"
    
    flights_df["arr_Rain"] = "NW"
    flights_df["arr_Fog"] = "NW"
    flights_df["arr_Snow"] = "NW"
    flights_df["arr_Cold"] = "NW"
    flights_df["arr_Storm"] = "NW"
    flights_df["arr_Hail"] = "NW"
    flights_df["arr_Precipitation"] = "NW"
    
    n = flights_df.shape[0]
    
    for i in range(n):
        temp_city = flights_df["origin_city"][i]
        departure_time = flights_df.at[i,"dep_datetime"]
        mask = (weather_df["LocalTimeStart"]<= departure_time) & (weather_df["LocalTimeEnd"] >= departure_time)
        weather_filtered_dep = weather_df[mask]
        weather_filtered_dep = weather_filtered_dep[weather_filtered_dep["City"] == temp_city]
        n_weather_temp = weather_filtered_dep.shape[0]
        if n_weather_temp != 0:
            for j in weather_filtered_dep.index:
                weather = weather_filtered_dep.at[j,"Type"]
                flights_df.at[i,"dep_" + weather] = weather_filtered_dep.at[j,"Severity"]
                
        temp_city = flights_df["dest_city"][i]
        arrival_time = flights_df.at[i,"arr_datetime"]
        mask = (weather_df["LocalTimeStart"]<= arrival_time) & (weather_df["LocalTimeEnd"] >= arrival_time)
        weather_filtered_dep = weather_df[mask]
        weather_filtered_dep = weather_filtered_dep[weather_filtered_dep["City"] == temp_city]
        n_weather_temp = weather_filtered_dep.shape[0]
        if n_weather_temp != 0:
            for j in weather_filtered_dep.index:
                weather = weather_filtered_dep.at[j,"Type"]
                flights_df.at[i,"arr_" + weather] = weather_filtered_dep.at[j,"Severity"]
    return(flights_df)

In [34]:
df_out = merge_flights_weather(flights_df, WeatherEvents)

In [35]:
df_out.iloc[0:40,-14:]

,dep_Rain,dep_Fog,dep_Snow,dep_Cold,dep_Storm,dep_Hail,dep_Precipitation,arr_Rain,arr_Fog,arr_Snow,arr_Cold,arr_Storm,arr_Hail,arr_Precipitation
0,NW,NW,NW,Severe,NW,NW,NW,NW,NW,NW,NW,NW,NW,NW
1,Light,NW,NW,Severe,NW,NW,NW,NW,NW,NW,NW,NW,NW,NW
2,NW,NW,NW,Severe,NW,NW,NW,NW,NW,NW,NW,NW,NW,NW
3,NW,NW,NW,Severe,NW,NW,NW,NW,NW,NW,NW,NW,NW,NW
4,Light,NW,NW,Severe,NW,NW,NW,NW,NW,NW,NW,NW,NW,NW
5,NW,Severe,NW,Severe,NW,NW,NW,NW,NW,NW,NW,NW,NW,NW
6,NW,NW,NW,Severe,NW,NW,NW,NW,NW,NW,NW,NW,NW,NW
7,NW,NW,NW,Severe,NW,NW,NW,NW,Moderate,NW,NW,NW,NW,NW
8,NW,NW,NW,Severe,NW,NW,NW,NW,NW,NW,NW,NW,NW,NW
9,NW,NW,NW,Severe,NW,NW,NW,NW,NW,NW,NW,NW,NW,NW


In [37]:
df_out["dep_Rain"].value_counts()

NW          135302
Light        12257
Moderate       828
Heavy           79
Name: dep_Rain, dtype: int64

In [38]:
df_out["dep_Fog"].value_counts()

NW          144352
Severe        3072
Moderate      1042
Name: dep_Fog, dtype: int64

In [39]:
df_out.to_csv("~/Desktop/DataDump/Flights_Weather_TEST.csv")

In [40]:
df_out.shape

(148466, 38)